## (1) 환경 준비

### 1) 라이브러리 불러오기

In [1]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np


### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [2]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv')
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv')

세 데이터프레임의 상위 5개 행을 조회하시오.

In [3]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [4]:
products.head()

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [5]:
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [7]:
tmp = sales.groupby('ProductID', as_index = False)['Qty','Amt'].sum()
tmp.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,ProductID,Qty,Amt
0,p1001771,1055,3354827
1,p1002841,903,11011541
2,p1005621,906,2601703
3,p1005771,3963,7319963
4,p1005891,5194,10119037


### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [9]:
tmp.sort_values('Amt', ascending = False).head(5)

,ProductID,Qty,Amt
22,p1072601,4058,18129067
43,p1178011,1653,14078818
1,p1002841,903,11011541
4,p1005891,5194,10119037
46,p1194801,990,7517664


### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [10]:
customers.groupby('Gender', as_index = False).count()

,Gender,CustomerID,RegisterDate,BirthYear
0,F,2060,2060,2060
1,M,183,183,183


### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [11]:
customers['Age'] = 2022 - customers['BirthYear']
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear,Age
0,c328222,2014-09-25,F,1960,62
1,c281448,2013-06-18,F,1974,48
2,c038336,2003-10-10,F,1968,54
3,c084237,2007-03-09,F,1982,40
4,c162600,2010-06-14,F,1978,44


### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40)  : 30 <= ,  < 40

In [15]:
bin = [0,30,40,50,60,70,100]
label = ['30미만','30대','40대','50대','60대','70이상']

customers['AgeGroup'] = pd.cut(customers['Age'], bins = bin , right=False, labels = label )
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,c328222,2014-09-25,F,1960,62,60대
1,c281448,2013-06-18,F,1974,48,40대
2,c038336,2003-10-10,F,1968,54,50대
3,c084237,2007-03-09,F,1982,40,40대
4,c162600,2010-06-14,F,1978,44,40대


### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [16]:
customers.groupby('AgeGroup', as_index = False).count()

,AgeGroup,CustomerID,RegisterDate,Gender,BirthYear,Age
0,30미만,6,6,6,6,6
1,30대,186,186,186,186,186
2,40대,925,925,925,925,925
3,50대,749,749,749,749,749
4,60대,267,267,267,267,267
5,70이상,110,110,110,110,110


### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [19]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
tmp = pd.merge(sales, products, on = 'ProductID')
tmp.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,순두부,반찬류,두부,1
1,137,4,2016-01-02,p1036481,2,2100,c280590,순두부,반찬류,두부,1
2,63,16,2016-01-03,p1036481,1,1050,c037915,순두부,반찬류,두부,1
3,135,3,2016-01-04,p1036481,3,3150,c100815,순두부,반찬류,두부,1
4,63,13,2016-01-06,p1036481,10,10500,c048405,순두부,반찬류,두부,1


In [21]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.
tmp2 = tmp.groupby('ProductName', as_index = False)['Amt'].sum()
tmp2.head()

,ProductName,Amt
0,감귤컵푸딩,677154
1,감자칩,3676518
2,고구마스틱,1620986
3,깻잎,2930359
4,낱개캔디,479197


In [22]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.
tmp2.sort_values('Amt', ascending = False).head(5)


,ProductName,Amt
37,우유1000,18129067
23,사과_소,14078818
9,딸기_대,11011541
7,두부_대,10119037
10,딸기_소,7517664


### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [23]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.
tmp = pd.merge(sales, customers, on = 'CustomerID')
tmp.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,48,40대
1,107,1,2016-01-02,p1175481,1,1300,c150417,2010-03-03,F,1974,48,40대
2,185,1,2016-01-04,p1162631,1,4600,c150417,2010-03-03,F,1974,48,40대
3,67,2,2016-01-11,p1012751,1,1350,c150417,2010-03-03,F,1974,48,40대
4,201,3,2016-01-12,p1005891,1,1950,c150417,2010-03-03,F,1974,48,40대


In [24]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.
tmp2 = tmp.groupby('AgeGroup', as_index = False)['Amt'].sum()
tmp2.head()

,AgeGroup,Amt
0,30미만,167539
1,30대,11730152
2,40대,86042546
3,50대,70047957
4,60대,19286478


### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [25]:
# 1) 세 데이터프레임을 merge 
tmp1 = pd.merge(sales, customers, on = 'CustomerID')
tmp2 = pd.merge(tmp1, products, on = 'ProductID')
tmp2.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,48,40대,순두부,반찬류,두부,1
1,197,5,2017-01-24,p1036481,1,1050,c150417,2010-03-03,F,1974,48,40대,순두부,반찬류,두부,1
2,81,5,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,39,30대,순두부,반찬류,두부,1
3,81,6,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,39,30대,순두부,반찬류,두부,1
4,23,13,2016-10-16,p1036481,1,1050,c350776,2015-06-06,F,1983,39,30대,순두부,반찬류,두부,1


In [37]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다
ag = tmp2['AgeGroup'].unique().sort_values()

for a in ag :
    tmp3 = tmp2.loc[tmp2['AgeGroup'] == a]
    t_amt = tmp3['Amt'].sum()
    tmp4 = tmp3.groupby('Category')['Amt'].sum() / t_amt
    print(tmp4)

Category
간식     0.080877
과일     0.088935
반찬류    0.310787
유제품    0.134631
채소     0.384770
Name: Amt, dtype: float64
Category
간식     0.068632
과일     0.264618
반찬류    0.129206
유제품    0.255305
채소     0.282240
Name: Amt, dtype: float64
Category
간식     0.083996
과일     0.257663
반찬류    0.138119
유제품    0.257045
채소     0.263177
Name: Amt, dtype: float64
Category
간식     0.057381
과일     0.256595
반찬류    0.171246
유제품    0.229995
채소     0.284783
Name: Amt, dtype: float64
Category
간식     0.034329
과일     0.226639
반찬류    0.242396
유제품    0.164646
채소     0.331990
Name: Amt, dtype: float64
Category
간식     0.025016
과일     0.279025
반찬류    0.269608
유제품    0.108956
채소     0.317394
Name: Amt, dtype: float64
